In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)   #entity
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [3]:
from TextSummariser.constants import *
from TextSummariser.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:  #configuration
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [17]:
from transformers import AutoTokenizer
from datasets import load_dataset
import os

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):

        # Ensure input is always a list of strings
        dialogues = example_batch['dialogue']
        summaries = example_batch['summary']

        # If a single string sneaks in, wrap it in a list
        if isinstance(dialogues, str):
            dialogues = [dialogues]
        if isinstance(summaries, str):
            summaries = [summaries]

        # Remove None values (if any)
        dialogues = [d if d is not None else "" for d in dialogues]
        summaries = [s if s is not None else "" for s in summaries]

        input_encodings = self.tokenizer(
            dialogues, max_length=1024, truncation=True, padding="max_length"
        )
        target_encodings = self.tokenizer(
            summaries, max_length=128, truncation=True, padding="max_length"
        )
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_dataset(
            "csv",
            data_files={
                "train": "artifacts/data_ingestion/samsum_dataset/train.csv",
                "validation": "artifacts/data_ingestion/samsum_dataset/validation.csv",
                "test": "artifacts/data_ingestion/samsum_dataset/test.csv"
            }
        )
        print(dataset_samsum["train"].column_names)  # Debug: check column names
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-06-25 22:40:41,053: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-25 22:40:41,056: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-25 22:40:41,062: INFO: common: created directory at: artifacts]
[2025-06-25 22:40:41,066: INFO: common: created directory at: artifacts/data_transformation]
['id', 'dialogue', 'summary']


Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 38780.03 examples/s]
